In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## VARIABLES

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Health/'
DATASET_PATH = BASE_PATH + 'dataset/'
VOCAB_PATH = '/content/drive/MyDrive/Health/vocab.txt'

TEST_SET_PATH = BASE_PATH + 'test_1000-1099.fas'

K = 6
SPLIT_SIZE = 3584 #100*671

# DNA BERT

[github link](https://github.com/jerryji1993/DNABERT)

## Install

### BioPython

In [ ]:
!pip install Bio

     |████████████████████████████████| 269 kB 9.7 MB/s 
     |████████████████████████████████| 2.3 MB 48.4 MB/s 


### DNA Bert Install

In [ ]:
!git clone https://github.com/jerryji1993/DNABERT

Cloning into 'DNABERT'...
remote: Enumerating objects: 766, done.
remote: Total 766 (delta 0), reused 0 (delta 0), pack-reused 766
Receiving objects: 100% (766/766), 11.60 MiB | 11.20 MiB/s, done.
Resolving deltas: 100% (403/403), done.


In [ ]:
%cd DNABERT
!pip install --editable .
%cd examples
!pip install -r requirements.txt

/content/DNABERT
Obtaining file:///content/DNABERT
     |████████████████████████████████| 5.7 MB 8.7 MB/s 
     |████████████████████████████████| 132 kB 51.6 MB/s 
     |████████████████████████████████| 1.2 MB 57.5 MB/s 
     |████████████████████████████████| 880 kB 42.3 MB/s 
     |████████████████████████████████| 8.8 MB 34.3 MB/s 
     |████████████████████████████████| 79 kB 9.3 MB/s 
     |████████████████████████████████| 138 kB 61.8 MB/s 
     |████████████████████████████████| 127 kB 50.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0cd31f3cc9de7f6e917626a26271f0e36158484a9f5f76954ae9c1fa7e950cc8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Running setup.py develop for t

## Imports


In [ ]:
from Bio import SeqIO
from itertools import product
import random
import glob
import os

## Vocab Building


In [ ]:
# lst = ["A", "C", "G", "T", "-"]
lst = ["A", "C", "G", "T"]
lengthOfStrings = 6
prod = product(lst, repeat=lengthOfStrings)
n_prod = 0
vocab = []
with open(VOCAB_PATH, 'w') as f:
  f.write('[PAD]\n[UNK]\n[CLS]\n[SEP]\n[MASK]\n')
  for i in prod:
    string = ''
    for letter in i:
      string += letter
    f.write(string + '\n')

## Sequences to Kmer

In [ ]:
def seq2kmer(seq, k, clean=True):
  if clean:
    seq = seq.replace('-','')
  kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
  kmers = " ".join(kmer)
  return kmers

def getLength(file_path):
  ctr = 0
  seq = SeqIO.parse(open(file_path),'fasta')
  for el in seq: ctr+=1
  return ctr

def getRandom():
  return random.uniform(0,1)

def seq2chars(seq, k, clean=True):
  if clean:
    seq = seq.replace('-','')
  mini_seq = ''
  for x in range(len(seq)):
    if x != 0 and x % k == 0:
      mini_seq += ' '
    mini_seq += seq[x]
          
  return mini_seq

# seq= '-----------G----GT---GTGATGGGGASACAJKSNCJSC'
# print(seq2kmer(seq, 6))

## Dataset

In [ ]:
!mkdir dataset
!mkdir output
!mkdir temp

### Training Set

In [ ]:
RATIO = 0.8
CARRIER_LABEL = 0
MENING_LABEL = 1

def getLabel(seq_header):
  label = CARRIER_LABEL
  if 'meningitis' in seq_header:
    label = MENING_LABEL
  return label

def buildTrainingSet(path, ratio=0.8):
  sequences = []
  for filename in glob.glob(path + 'ms_*.fas'):
    print(filename)
    with open(filename) as ms_in:
      sequences += ms_in.readlines()

  for filename in glob.glob(path + 'c_*.fas'):
    print(filename)
    with open(filename) as ms_in:
      sequences += ms_in.readlines()

  with open('dataset.fas', 'w') as seq_file:
    seq_file.write(''.join(sequences))

  parsed_seq = SeqIO.parse(open('dataset.fas'),'fasta')
  train_ctr = 0
  dev_ctr = 0 

  os.makedirs('./temp/', exist_ok=True)
  with open('./temp/train.tsv', 'w') as train_file:
    with open('./temp/dev.tsv', 'w') as dev_file:
      train_file.write('sequence\tlabel\n')
      dev_file.write('sequence\tlabel\n')

      for seq_record in parsed_seq:
        label = 0
        if 'meningitis' in seq_record.id:
          label = 1
        kmered = seq2kmer(str(seq_record.seq), K)
        if kmered:
          if getRandom() < ratio:
            train_file.write(kmered + '\t' + str(label) + '\n')
            train_ctr += 1
          else:
            dev_file.write(kmered + '\t' + str(label) + '\n')
            dev_ctr += 1
      print('TOTAL: ' + str(train_ctr + dev_ctr))
      print('\t Train: ' + str(train_ctr))
      print('\t Dev: ' + str(dev_ctr))

In [ ]:
buildTrainingSet(DATASET_PATH)

/content/drive/MyDrive/Health/dataset/ms_001-299.fas
/content/drive/MyDrive/Health/dataset/ms_300-599.fas
/content/drive/MyDrive/Health/dataset/ms_600-899.fas
/content/drive/MyDrive/Health/dataset/c_001-299.fas
/content/drive/MyDrive/Health/dataset/c_300-599.fas
/content/drive/MyDrive/Health/dataset/c_600-899.fas
TOTAL: 1779
	 Train: 1418
	 Dev: 361


In [ ]:
!cp temp/* /content/drive/MyDrive/Health/dataset/temp/

### OLD Training Set

In [ ]:
# Open ms_data in write mode
with open('ms_data.fas', 'w') as outfile:
  filenames = glob.glob(DATASET_PATH + 'ms_*.fas')
  for names in filenames:
    print(names)
    with open(names) as infile:
      outfile.write(infile.read())
    # outfile.write("\n")

with open('c_data.fas', 'w') as outfile:
  filenames = glob.glob(DATASET_PATH + 'c_*.fas')
  for names in filenames:
    print(names)
    with open(names) as infile:
      outfile.write(infile.read())
    # outfile.write("\n")

/content/drive/MyDrive/Health/dataset/ms_001-299.fas
/content/drive/MyDrive/Health/dataset/ms_300-599.fas
/content/drive/MyDrive/Health/dataset/ms_600-899.fas
/content/drive/MyDrive/Health/dataset/c_001-299.fas
/content/drive/MyDrive/Health/dataset/c_300-599.fas
/content/drive/MyDrive/Health/dataset/c_600-899.fas


In [ ]:
# mening_sequences = SeqIO.parse(open('ms_data.fas'),'fasta')
# for seq_record in mening_sequences:
#   print(seq_record.id.split('|')[1])
  # print(repr(seq_record.seq))
  # print(len(seq_record))
  
def getLength(file_path):
  ctr = 0
  seq = SeqIO.parse(open(file_path),'fasta')
  for el in seq: ctr+=1
  return ctr

# Writing Training and Validation sets using a 9/1 ratio
mening_sequences = SeqIO.parse(open('ms_data.fas'),'fasta')
carrier_sequences = SeqIO.parse(open('c_data.fas'),'fasta')

DATASET_LENGTH = getLength('ms_data.fas')
RATIO = 0.9
LIMIT = int(RATIO*DATASET_LENGTH)
print(LIMIT)

CARRIER_LABEL = 0
MENING_LABEL = 1

810


In [ ]:
with open('./dataset/train.tsv', 'w') as t_file:
  with open('./dataset/dev.tsv', 'w') as d_file:
    ctr = 0
    t_file.write('sequence\tlabel\n')
    d_file.write('sequence\tlabel\n')
    for seq_record in mening_sequences:
      kmered = seq2kmer(str(seq_record.seq), K)
      if not kmered: continue
      if ctr < LIMIT:
        t_file.write(kmered + '\t' + str(MENING_LABEL) + '\n')
      else:
        d_file.write(kmered + '\t' + str(MENING_LABEL) + '\n')
      ctr += 1
    
    ctr = 0
    for seq_record in carrier_sequences:
      kmered = seq2kmer(str(seq_record.seq), K)
      if not kmered: continue
      if ctr < LIMIT:
        t_file.write(kmered + '\t' + str(CARRIER_LABEL) + '\n')
      else:
        d_file.write(kmered + '\t' + str(CARRIER_LABEL) + '\n')
      ctr += 1

print('Sequenced ' + str(ctr) + ' samples.')

Sequenced 900 samples.


### Test Set

In [ ]:
!ls $TEST_SET_PATH

/content/drive/MyDrive/Health/test_1000-1099.fas


In [ ]:
def buildTestSet(path):
  test_sequences = SeqIO.parse(open(path),'fasta')
  os.makedirs('./temp/', exist_ok=True)
  with open('./temp/test.tsv', 'w') as test_file:
    test_file.write('sequence\tlabel\n')
    for seq_record in test_sequences:
      kmered = seq2kmer(str(seq_record.seq), K)
      label = 1
      if 'carrier' in seq_record.id:
        label = 0
      test_file.write(kmered + '\t' + str(label) + '\n')

In [ ]:
buildTrainingSet(TEST_SET_PATH)

TOTAL: 0
	 Train: 0
	 Dev: 0


### Old Test Set

In [ ]:
test_sequences = SeqIO.parse(open(TEST_SET_PATH),'fasta')

with open('./dataset/test.tsv', 'w') as test_file:
  test_file.write('sequence\tlabel\n')
  for seq_record in test_sequences:
    kmered = seq2kmer(str(seq_record.seq), K)
    label = getLabel(seq_record.id)
    test_file.write(kmered + '\t' + str(label) + '\n')

In [ ]:
# !cp ./dataset/test.tsv /content/drive/MyDrive/Health/dataset/test.tsv
!cp ./temp/test.tsv /content/drive/MyDrive/Health/dataset/temp/test.tsv

### Sequences Split and Shuffle

In [ ]:
filenames = ['train.tsv', 'dev.tsv', 'test.tsv']

def splitLine(seq, label, split_size):
  with open('temp/' + filename, 'r') as f:
    with open('dataset/' + filename, 'w') as out:
      print(filename)
      lines = f.readlines()
      out.write(lines[0])
      lines = lines[1:]
      random.shuffle(lines)
      ctr = 0

      for line in lines:
        start_pos = 0
        end_pos = split_size
        line = line.split('\t')
        while(start_pos<=len(line[0])):
          out.write(line[0][start_pos:end_pos] + '\t' + str(ctr))
          start_pos+=split_size
          end_pos+=split_size
        ctr += 1
  

def splitSequence(filename, split_size=SPLIT_SIZE):
  with open('temp/' + filename, 'r') as f:
    with open('dataset/' + filename, 'w') as out:
      print(filename)
      lines = f.readlines()
      out.write(lines[0])
      lines = lines[1:]
      random.shuffle(lines)
      ctr = 0
      for line in lines:
        if ctr != len(lines) - 1:
          line = line.split('\t')
          out.write(line[0][0:split_size] + '\t' + line[1])
        else:
          line = line.split('\t')
          out.write(line[0][0:split_size] + '\t' + line[1].split('\n')[0])
        ctr += 1


def count_lines(filename):
  f = open(filename)                  
  lines = 0
  buf_size = 1024 * 1024
  read_f = f.read # loop optimization

  buf = read_f(buf_size)
  while buf:
      lines += buf.count('\n')
      buf = read_f(buf_size)

  return lines

def splitSeq2Lines(filename, split_size):
  os.makedirs('./dataset/', exist_ok=True)
  in_path = DATASET_PATH + 'temp/'  + filename
  out_path = 'dataset/' + filename

  with open(in_path, 'r') as f:
    n_lines = count_lines(in_path)
    print('LINES: ' + str(n_lines))
    with open(out_path, 'w') as out:
      print(filename)
      
      line = f.readline()
      out.write('sequence\tid\n') # writing header
      ctr = 0
      while line:
        # do stuff with line
        line = f.readline()
        start_pos = 0
        end_pos = split_size
        splitted_line = line.split('\t')
        while(start_pos<=len(splitted_line[0])):
          out.write(splitted_line[0][start_pos:end_pos-1] + '\t' + str(ctr) + '\n')
          start_pos+=split_size
          end_pos+=split_size
        ctr += 1
        if ctr % 100 == 0: 
          print('['+filename+'] ' + str(ctr/n_lines*100) + '%\t' + str(ctr))

SPLIT_SIZE = 3584 #512 words (kmers)
print('Split Size: ' + str(SPLIT_SIZE))
for filename in filenames:
  splitSeq2Lines(filename, SPLIT_SIZE)

In [ ]:
count_lines(DATASET_PATH + 'dataset_k512/train/train.tsv')

1198368

In [ ]:
# copying test dataset to its oown folder and renaming it because they are stupid and didn't think about it
!mkdir test
!mv dataset/test.tsv test/dev.tsv

mkdir: cannot create directory ‘test’: File exists


In [ ]:
# !cp prova/test.tsv test/dev.tsv

In [ ]:
# DEST_PATH = DATASET_PATH + 'dataset_300M_300C.txt'
# !cp dataset_300M_300C.txt $DEST_PATH

### Get ID-Labels

In [ ]:
TEMP_DATASET = "/content/drive/MyDrive/Health/dataset/temp/"
!ls $TEMP_DATASET

dev.tsv  test.tsv  train.tsv


In [ ]:
def generateIdLabelsFile(filename):
  output_path = filename.split('.')[0] + '_id_labels.txt'
  with open(TEMP_DATASET + filename, 'r') as input, open(output_path, 'w') as output:
    n_lines = count_lines(TEMP_DATASET + filename)
    print('[' + filename + ']LINES: ' + str(n_lines))
    line = input.readline() #header
    output.write('seq_id\tlabel\n') #write header
    ctr = 0
    while line:
      line = input.readline()
      splitted_line = line.split('\t')
      if len(splitted_line) < 2:
        break
      label = splitted_line[1]
      seq_id = ctr
      output.write(str(seq_id) + '\t' + str(label))
      ctr += 1
      if ctr % 100 == 0: 
        print(ctr)
  print('File saved: ' + output_path)

In [ ]:
filename = 'test.tsv'
generateIdLabelsFile(filename)

[test.tsv]LINES: 201
100
200
File saved: test_id_labels.txt


In [ ]:
!more test_id_labels.txt

seq_id	label
0	0
1	0
2	0
3	0
4	0
5	0
6	0
7	0
8	0
9	0
10	0
11	0
12	0
13	0
14	0
15	0
16	0
17	0
18	0
19	0
20	0
21	0


In [ ]:
!mkdir emb_test
!unzip /content/drive/MyDrive/Health/dataset/dataset_k512/embeddings/test.zip
!mv *.pt ./emb_test/

In [ ]:
import glob
import torch

def load_id_labels(path):
  labels = []
  with open(path, 'r') as input:
    line = input.readline()
    while line:
      line = input.readline()
      splitted_line = line.split('\t')
      if len(splitted_line) < 2:
        break
      seq_id = splitted_line[0]
      label = int(splitted_line[1])
      labels.append(label)

  return labels

labels = load_id_labels('test_id_labels.txt')
print(len(labels))

def getEmbeddingDataset(path):
  embeddings = {}
  emb_array = []
  for filename in sorted(glob.glob(path + '/*.pt')):
    seq_id = int(filename.split('.pt')[0].split('seq')[1])
    if seq_id == len(labels): # erase the last one since it's empty
      break
    label = labels[seq_id]
    tensors = torch.load(filename, map_location=torch.device('cpu'))
    embeddings[seq_id] = {'label': label, 
                          'tensors': tensors
                          }
    emb_array.append(tensors)
  return embeddings, emb_array
  # print('seq[' + str(seq_id) + '] label = ' + str(label))

embeddings, emb_array = getEmbeddingDataset('emb_test')

200


In [ ]:
for emb in emb

NameError: ignored

In [ ]:
train_X = []
train_y = []
for i in range(len(emb_array)):
  label = labels[i]
  for emb in emb_array[i].values():
    train_X.append(emb.tolist())
    train_y.append(label)

import pandas as pd
df = pd.DataFrame(list(zip(train_X, train_y)),
               columns =['X', 'y'])

In [ ]:
df.head()

,X,y
0,"[0.5868362188339233, -0.2806219160556793, -0.3...",0
1,"[0.21693071722984314, -0.1528603583574295, 0.2...",0
2,"[0.586708664894104, -0.2802076041698456, 0.030...",0
3,"[0.25666671991348267, -0.54011070728302, 0.551...",0
4,"[0.5566396713256836, -0.2230125218629837, 0.13...",0


In [ ]:
from operator import itemgetter
indices = [x for x in range(768)]
df = df['X'].transform({f'X{i+1}': itemgetter(i) for i in indices})
df['y'] = train_y 

In [ ]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X760,X761,X762,X763,X764,X765,X766,X767,X768,y
0,0.586836,-0.280622,-0.393021,-0.006847,0.532320,0.069348,0.587181,-0.426089,-0.044642,-0.144710,...,0.288943,-0.177624,0.310271,-0.602017,-0.116125,-0.100209,-0.293820,0.382296,0.270869,0
1,0.216931,-0.152860,0.269706,0.317533,0.560679,-0.449366,0.103354,-0.144298,0.240491,0.313215,...,0.218332,-0.106709,-0.782730,-0.144598,-0.387436,-0.037106,0.074231,0.504337,0.384010,0
2,0.586709,-0.280208,0.030081,-0.115715,0.466501,0.442316,-0.454067,-0.302339,-0.062956,0.206698,...,-0.751148,0.193287,-0.677578,-0.275258,-0.447550,-0.566069,-0.354516,0.297212,0.097665,0
3,0.256667,-0.540111,0.551641,-0.362935,0.050804,-0.479595,0.234828,-0.463642,0.251334,0.516325,...,-0.106587,0.228359,-0.533587,-0.031965,-0.422599,0.416044,0.322401,0.497008,0.092302,0
4,0.556640,-0.223013,0.138141,-0.607483,0.066755,-0.385463,0.162250,0.415066,-0.003597,-0.104311,...,-0.032077,0.408256,-0.017650,-0.267663,-0.398181,0.408359,0.703377,0.369062,-0.052011,0


In [ ]:
y = df[df.columns[-1:]]
X = df[df.columns[:-1]]

X = X.values
y = y.values

print('X: ({},{})'.format(X.shape[0], X.shape[1]))
print('y: ({},{})'.format(y.shape[0], y.shape[1]))

X: (31034,768)
y: (31034,1)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
scaler = scaler.fit(X)

tr_X, ts_X, tr_y, ts_y = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

rfc = RandomForestClassifier()
rfc.fit(tr_X,tr_y.ravel())

RandomForestClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

# predictions
rfc_predict = rfc.predict(ts_X)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(ts_y, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(ts_y, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 

=== Confusion Matrix ===
[[ 702 2132]
 [ 798 2575]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.47      0.25      0.32      2834
           1       0.55      0.76      0.64      3373

    accuracy                           0.53      6207
   macro avg       0.51      0.51      0.48      6207
weighted avg       0.51      0.53      0.49      6207



=== All AUC Scores ===
[0.43868739 0.49816939 0.50829459 0.48519542 0.49946919 0.50354878
 0.48769667 0.4814686  0.48423051 0.50005319]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.4886813727265105


In [ ]:
# import tensorflow.keras.backend as K
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier

# from sklearn.model_selection import KFold

# forest = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = forest, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
# rf_random.fit(df.X, df.y)

## Finetuning

In [ ]:
KMER = K
MODEL_PATH=BASE_PATH + 'DNABERT6/'
DATA_PATH='dataset'
OUTPUT_PATH='output'

EPOCHS = 5.0

In [ ]:
!python DNABERT/examples/run_finetune.py \
    --model_type dna \
    --tokenizer_name=dna$KMER \
    --model_name_or_path $MODEL_PATH \
    --task_name dnaprom \
    --do_train \
    --do_eval \
    --data_dir $DATA_PATH \
    --max_seq_length 100 \
    --per_gpu_eval_batch_size=32   \
    --per_gpu_train_batch_size=32   \
    --learning_rate 2e-4 \
    --num_train_epochs $EPOCHS \
    --output_dir $OUTPUT_PATH \
    --evaluate_during_training \
    --logging_steps 100 \
    --save_steps 4000 \
    --warmup_percent 0.1 \
    --hidden_dropout_prob 0.1 \
    --overwrite_output \
    --weight_decay 0.01 \
    --n_process 8

05/09/2022 10:53:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/09/2022 10:53:04 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/MyDrive/Health/DNABERT6/config.json
05/09/2022 10:53:04 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": "dnaprom",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_b

In [ ]:
# 05/03/2022 11:28:33 - INFO - __main__ -   ***** Eval results  *****
# 05/03/2022 11:28:33 - INFO - __main__ -     acc = 0.9833333333333333
# 05/03/2022 11:28:33 - INFO - __main__ -     auc = 0.9916666666666666
# 05/03/2022 11:28:33 - INFO - __main__ -     f1 = 0.983328702417338
# 05/03/2022 11:28:33 - INFO - __main__ -     mcc = 0.9672041516493516
# 05/03/2022 11:28:33 - INFO - __main__ -     precision = 0.9838709677419355
# 05/03/2022 11:28:33 - INFO - __main__ -     recall = 0.9833333333333334

## Test

In [ ]:
KMER = K
MODEL_PATH='output'
DATA_PATH='test'
PREDICTION_PATH='prediction'

In [ ]:
!python DNABERT/examples/run_finetune.py \
    --model_type dna \
    --tokenizer_name=dna$KMER \
    --model_name_or_path $MODEL_PATH \
    --task_name dnaprom \
    --do_predict \
    --data_dir $DATA_PATH  \
    --max_seq_length 75 \
    --per_gpu_pred_batch_size=128   \
    --output_dir $MODEL_PATH \
    --predict_dir $PREDICTION_PATH \
    --n_process 48

05/09/2022 11:01:43 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/09/2022 11:01:43 - INFO - transformers.configuration_utils -   loading configuration file output/config.json
05/09/2022 11:01:43 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": "dnaprom",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_h

## Remove cache

In [ ]:
!rm -r test/cached*
!rm -r dataset/cached*

rm: cannot remove 'test/cached*': No such file or directory
rm: cannot remove 'dataset/cached*': No such file or directory


## Get IDs

In [ ]:
START = 300
N_SAMPLES = 300

CARRIER = BASE_PATH + "ids_carrier.txt"
MENING_SEPT = BASE_PATH + "ids_meningitis_septicaemia.txt"

def getIds(path=CARRIER, id_start=START, n_samples=N_SAMPLES):
  my_file = open(path, "r")
  content_list = my_file.readlines()
  lista = [int(x) for x in content_list[id_start:id_start+n_samples]]
  # for n in lista:
  #   print(n)
  return lista

carrier_set = getIds(CARRIER, id_start=1000, n_samples=100)
mening_set = getIds(MENING_SEPT, id_start=1000, n_samples=100)
test_set = carrier_set + mening_set
random.shuffle(test_set)

In [ ]:
for el in test_set:
  print(el)

2195
92282
92771
92091
92346
2172
92264
92108
2277
2240
92237
92381
2177
92110
92231
92017
92156
2161
2160
92142
92307
2194
92377
92035
2175
2278
92096
2186
92049
92451
2205
2171
92763
92308
2158
92173
92241
92155
92759
2169
2215
2153
2174
92403
92065
2189
92078
2199
92461
92425
2182
92080
92086
2187
2168
2178
92199
2234
92860
2188
2151
92141
92330
92211
92187
2159
92212
2190
92140
2200
2197
92064
2254
92414
2279
92068
2179
92234
2241
2236
2235
2176
2180
2223
92364
92321
2261
92340
2183
92376
92453
2184
2256
2226
2263
92221
92284
92290
92137
2193
2191
92271
2265
92188
2258
92447
92196
92341
2232
2225
2210
2239
92112
2162
2259
2152
2202
2208
2255
92194
2229
92227
2154
92168
2228
92356
92191
92280
92448
2231
2260
2185
2173
2238
92242
2166
92025
92430
2163
2167
2217
2164
92217
92164
2230
2262
2165
92384
92441
92040
2203
92088
92131
92329
2201
92292
92223
2196
92395
2209
92092
92053
2221
2266
2207
2224
92435
92159
2257
2155
92081
2192
92300
2181
2204
92028
2227
2211
92098
2213
92844
92281


In [ ]:
ids = getIds(CARRIER, id_start=900)
for el in ids:
  print(el)

2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2207
2208
2209
2210
2211
2213
2215
2217
2218
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2234
2235
2236
2238
2239
2240
2241
2242
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2277
2278
2279
